In [1]:
import pandas as pd
from sklearn.model_selection import cross_val_predict, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler

C:\Users\ugurs\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
total_data = pd.read_csv('C:/Users/ugurs/Python Workspace Laptop/Migros Workspace/GSU_MIGROS/train_data_7.csv', sep = ';')

In [3]:
total_data.head()

,MalNo,SatisMiktar,SatisNetTutar,TarihIndex,StokMiktar,ImhaMiktar,KategoriAnaGrupNo,KategoriGrupNo,KategoriSinifNo,KategoriAltSinifNo,...,Yil,DayOfYear,DayOfWeek,MonthStart,Weekend,YilBasinaUzaklik,GunlukSatisOrtalamasi,GunlukSatisMedian,GunlukSatisOrt2016,GunlukSatisMed2016
0,12284590,2.0,35.59,2014-01-01,1.0,0.0,300,10,11,10,...,2014,1,2,1,0,364.0,0.25,0.0,1.0,1.0
1,12284590,2.0,62.55,2014-12-24,2.0,0.0,300,10,11,10,...,2014,358,2,0,0,7.0,0.25,0.0,1.0,1.0
2,12284590,3.0,101.43,2015-12-09,14.0,0.0,300,10,11,10,...,2015,343,2,0,0,22.0,0.25,0.0,1.0,1.0
3,12284590,1.0,33.81,2016-12-21,4.0,0.0,300,10,11,10,...,2016,356,2,0,0,10.0,0.25,0.0,1.0,1.0
4,12284590,1.0,38.05,2016-12-28,3.0,0.0,300,10,11,10,...,2016,363,2,0,0,3.0,0.25,0.0,1.0,1.0


In [4]:
df = total_data[['MalNo', 'SatisMiktar', 'SatisNetTutar', 'ImhaMiktar']].groupby('MalNo').sum()

In [5]:
df_imhaOlan = df[(df['ImhaMiktar'] > 0) & (df['SatisMiktar'] > 0)]

In [6]:
df_imhaOlan.sort_values('ImhaMiktar', ascending = False).head()

,SatisMiktar,SatisNetTutar,ImhaMiktar
MalNo,,,
70280008,158829.779,628371.13,23397.916
280008,140767.232,382795.61,19547.855
280017,42206.043,152596.73,18962.940
280019,26269.066,86736.80,16399.790
280016,79702.539,228881.49,15192.707


In [7]:
total_data[total_data['MalNo'] == 280008].sort_values('TarihIndex').head()

,MalNo,SatisMiktar,SatisNetTutar,TarihIndex,StokMiktar,ImhaMiktar,KategoriAnaGrupNo,KategoriGrupNo,KategoriSinifNo,KategoriAltSinifNo,...,Yil,DayOfYear,DayOfWeek,MonthStart,Weekend,YilBasinaUzaklik,GunlukSatisOrtalamasi,GunlukSatisMedian,GunlukSatisOrt2016,GunlukSatisMed2016
184537,280008,1.034,2.87,2014-04-16,0.000,0.0,300,15,25,44,...,2014,106,2,0,0,259.0,131.972766,120.4975,130.834462,116.7125
185426,280008,0.000,0.00,2014-04-17,-1.034,0.0,300,15,25,44,...,2014,107,3,0,0,258.0,137.029648,128.9905,142.311269,128.9905
184789,280008,0.000,0.00,2014-04-18,-1.034,0.0,300,15,25,44,...,2014,108,4,0,0,257.0,150.214891,139.3255,158.472132,138.5500
184917,280008,0.000,0.00,2014-04-19,-1.034,0.0,300,15,25,44,...,2014,109,5,0,0,256.0,185.488656,180.3260,190.724528,180.0500
185044,280008,0.000,0.00,2014-04-20,-1.034,0.0,300,15,25,44,...,2014,110,6,0,1,255.0,194.064531,176.5330,198.758788,175.3525


In [8]:
mal_280008 = total_data[total_data['MalNo'] == 280008] 
mal_280008_sort = mal_280008.sort_values('TarihIndex')
mal_280008_sort.set_index(['MalNo', 'TarihIndex'], inplace = True)

In [9]:
mal_280008_sort_ld = mal_280008_sort[['StokMiktar','SatisMiktar',
                 'ImhaMiktar']].shift(1).rename(columns = {'StokMiktar': 'StokMiktar_ld',
                                                                                      'SatisMiktar' : 'SatisMiktar_ld',
                                                                                      'ImhaMiktar' : 'ImhaMiktar_ld'})

In [10]:
mal_280008_sort_ld.fillna(0, inplace = True)

In [11]:
mal_280008_2 = mal_280008_sort.merge(mal_280008_sort_ld, how = 'inner', on = ['MalNo', 'TarihIndex'])

In [12]:
mal_280008_2['YeniUrunGelmisMi'] = (mal_280008_2['StokMiktar_ld'] - (
    mal_280008_2['SatisMiktar'] + mal_280008_2['ImhaMiktar']) < mal_280008_2['StokMiktar'])

In [13]:
_280008 = mal_280008_2.reset_index().drop(['MalNo','KategoriAnaGrupNo','KategoriGrupNo',
                                           'KategoriSinifNo','KategoriAltSinifNo','KategoriTamami',
                                           'KategoriSinifTamami','UrunSayisi', 'UrunSinifSayisi'], axis = 1)

MalNo = 280008 i düşürdüm hepsi aynı olduğu için tek üründe

In [14]:
_280008.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 894 entries, 0 to 893
Data columns (total 22 columns):
TarihIndex               894 non-null object
SatisMiktar              894 non-null float64
SatisNetTutar            894 non-null float64
StokMiktar               894 non-null float64
ImhaMiktar               894 non-null float64
Gun                      894 non-null int64
Ay                       894 non-null int64
Ceyrek                   894 non-null int64
Yil                      894 non-null int64
DayOfYear                894 non-null int64
DayOfWeek                894 non-null int64
MonthStart               894 non-null int64
Weekend                  894 non-null int64
YilBasinaUzaklik         894 non-null float64
GunlukSatisOrtalamasi    894 non-null float64
GunlukSatisMedian        894 non-null float64
GunlukSatisOrt2016       894 non-null float64
GunlukSatisMed2016       894 non-null float64
StokMiktar_ld            894 non-null float64
SatisMiktar_ld           894 non-null 

In [15]:
_280008_Yil_Ay_dw = _280008.groupby(['Yil','Ay', 'DayOfWeek'])['SatisMiktar','SatisNetTutar'].sum()

In [16]:
_280008_Yil_Ay_dw.head(7)

SatisMiktar  SatisNetTutar
Yil  Ay DayOfWeek                            
2014 4  0                0.000           0.00
        1                0.000           0.00
        2                1.034           2.87
        3                0.000           0.00
        4                0.000           0.00
        5                0.000           0.00
        6                0.000           0.00

In [17]:
_280008_Yil_Ay_dw['BirimFiyat'] = _280008_Yil_Ay_dw['SatisNetTutar'].divide(
    _280008_Yil_Ay_dw['SatisMiktar'])

In [18]:
_280008_Yil_Ay_dw.groupby(['Yil', 'Ay']).mean()

SatisMiktar  SatisNetTutar  BirimFiyat
Yil  Ay                                        
2014 4      0.147714       0.410000    2.775629
     7      0.000000       0.000000         NaN
     8    258.380429     667.098571    2.569395
     9    844.316000    2050.254286    2.425290
     10   992.128143    3155.170000    3.180653
     11   754.757571    2139.567143    2.839412
     12   519.184571    1365.950000    2.635262
2015 1    386.744429    1177.178571    3.063920
     2    449.536143    1303.938571    2.905223
     3    535.096143    1568.445714    2.977195
     4    511.737714    2150.904286    4.192476
     5    662.224429    2373.525714    3.622337
     6    770.966000    1419.860000    1.843820
     7    711.401143    1672.632857    2.348578
     8   1009.350000    1655.441429    1.656434
     9   1037.164571    2351.268571    2.262591
     10  1062.296000    2625.028571    2.471890
     11   684.035571    2160.375714    3.166942
     12   486.908143    1936.177143    3.990348
2016 1    466.312857    1953.180000    4.206698
     2    490.448143    1071.931429    2.177303
     3    623.016571    1393.528571    2.241743
     4    606.615571    2078.760000    3.448514
     5    588.826857    1244.440000    2.091142
     6    739.328857    1512.207143    2.070586
     7    781.875571    2593.960000    3.336362
     8    715.903286    1858.582857    2.593258
     9   1203.595286    3548.498571    2.955323
     10  1135.491857    3022.087143    2.691475
     11   620.643714    1420.445714    2.296683
     12   461.171286    1214.238571    2.630698
2017 1      0.000000       0.000000         NaN

In [19]:
_280008_index_r = _280008_Yil_Ay_dw.reset_index()

In [20]:
_280008_index_r[(_280008_index_r['Yil'] == 2014 ) & (_280008_index_r['Ay'] == 4 )]

,Yil,Ay,DayOfWeek,SatisMiktar,SatisNetTutar,BirimFiyat
0,2014,4,0,0.000,0.00,NaN
1,2014,4,1,0.000,0.00,NaN
2,2014,4,2,1.034,2.87,2.775629
3,2014,4,3,0.000,0.00,NaN
4,2014,4,4,0.000,0.00,NaN
5,2014,4,5,0.000,0.00,NaN
6,2014,4,6,0.000,0.00,NaN


2014 ün 9. ayından itibaren aldım (gerekirse 2015 ten başlat veriyi)

In [21]:
_280008_201409 = _280008[(_280008['TarihIndex'] >= '2014-09-01') & (_280008['TarihIndex'] < '2017-01-01') ]

In [22]:
(_280008_201409['SatisMiktar'] == 0).sum()

0

In [23]:
_280008_201409['BirimFiyat'] = _280008_201409['SatisNetTutar'].divide(_280008_201409['SatisMiktar'])

C:\Users\ugurs\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [24]:
_280008_201409.head()

,TarihIndex,SatisMiktar,SatisNetTutar,StokMiktar,ImhaMiktar,Gun,Ay,Ceyrek,Yil,DayOfYear,...,YilBasinaUzaklik,GunlukSatisOrtalamasi,GunlukSatisMedian,GunlukSatisOrt2016,GunlukSatisMed2016,StokMiktar_ld,SatisMiktar_ld,ImhaMiktar_ld,YeniUrunGelmisMi,BirimFiyat
40,2014-09-01,215.601,596.90,151.899,0.0,1,9,3,2014,244,...,121.0,166.400669,150.2930,169.208154,142.5760,-163.663,258.907,0.0,True,2.768540
41,2014-09-02,171.275,474.12,150.124,0.0,2,9,3,2014,245,...,120.0,136.942709,124.9070,138.217827,120.1030,151.899,215.601,0.0,True,2.768180
42,2014-09-03,160.815,445.30,224.809,66.0,3,9,3,2014,246,...,119.0,131.972766,120.4975,130.834462,116.7125,150.124,171.275,0.0,True,2.769020
43,2014-09-04,145.243,362.50,229.566,0.0,4,9,3,2014,247,...,118.0,137.029648,128.9905,142.311269,128.9905,224.809,160.815,66.0,True,2.495817
44,2014-09-05,146.700,365.36,-5.134,0.0,5,9,3,2014,248,...,117.0,150.214891,139.3255,158.472132,138.5500,229.566,145.243,0.0,False,2.490525


In [25]:
_280008_201409.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 853 entries, 40 to 892
Data columns (total 23 columns):
TarihIndex               853 non-null object
SatisMiktar              853 non-null float64
SatisNetTutar            853 non-null float64
StokMiktar               853 non-null float64
ImhaMiktar               853 non-null float64
Gun                      853 non-null int64
Ay                       853 non-null int64
Ceyrek                   853 non-null int64
Yil                      853 non-null int64
DayOfYear                853 non-null int64
DayOfWeek                853 non-null int64
MonthStart               853 non-null int64
Weekend                  853 non-null int64
YilBasinaUzaklik         853 non-null float64
GunlukSatisOrtalamasi    853 non-null float64
GunlukSatisMedian        853 non-null float64
GunlukSatisOrt2016       853 non-null float64
GunlukSatisMed2016       853 non-null float64
StokMiktar_ld            853 non-null float64
SatisMiktar_ld           853 non-null

In [26]:
train_data_1 = _280008_201409[_280008_201409['TarihIndex'] < '2016-10-01']

In [27]:
train_data_2 = train_data_1.drop(['TarihIndex','SatisNetTutar','StokMiktar','ImhaMiktar'], axis = 1)

In [28]:
test_data_1 = _280008_201409[_280008_201409['TarihIndex'] >= '2016-10-01']

In [29]:
test_data_2 = test_data_1.drop(['TarihIndex','SatisNetTutar','StokMiktar','ImhaMiktar'], axis = 1)

In [35]:
X = test_data_2.iloc[:,1:]

In [40]:
y = test_data_2.iloc[:,0:1]

In [44]:
X.head()

,Gun,Ay,Ceyrek,Yil,DayOfYear,DayOfWeek,MonthStart,Weekend,YilBasinaUzaklik,GunlukSatisOrtalamasi,GunlukSatisMedian,GunlukSatisOrt2016,GunlukSatisMed2016,StokMiktar_ld,SatisMiktar_ld,ImhaMiktar_ld,YeniUrunGelmisMi,BirimFiyat
801,1,10,4,2016,275,5,1,0,91.0,185.488656,180.3260,190.724528,180.0500,145.682,283.898,38.99,True,2.409324
802,2,10,4,2016,276,6,0,1,90.0,194.064531,176.5330,198.758788,175.3525,182.413,332.587,0.00,True,2.398369
803,3,10,4,2016,277,0,0,0,89.0,166.400669,150.2930,169.208154,142.5760,104.049,388.364,0.00,True,2.398396
804,4,10,4,2016,278,1,0,0,88.0,136.942709,124.9070,138.217827,120.1030,309.227,375.772,0.00,True,2.398246
805,5,10,4,2016,279,2,0,0,87.0,131.972766,120.4975,130.834462,116.7125,56.805,252.422,25.05,True,1.628440


In [30]:
def rfr_model(X, y):

# Perform Grid-Search
    gsc = GridSearchCV(
        estimator=RandomForestRegressor(),
        param_grid={            
            'max_depth': range(3,7),
            'n_estimators': (10, 50, 100, 1000),},
        cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
        
    grid_result = gsc.fit(X, y)
    best_params = grid_result.best_params_
    
    rfr = RandomForestRegressor(max_depth=best_params["max_depth"], n_estimators=best_params["n_estimators"],
                                random_state=False, verbose=False)
    
# Perform K-Fold CV
    scores = cross_val_predict(rfr, X, y, cv=10, scoring='neg_mean_absolute_error')
    return scores

In [45]:
rfr_model(X,y)

C:\Users\ugurs\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:740: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


TypeError: cross_val_predict() got an unexpected keyword argument 'scoring'